# Periodic frequent-pattern discovery
Using algorithm from this paper: https://www.researchgate.net/publication/356825094_Discovering_Periodic-Frequent_Patterns_in_Uncertain_Temporal_Databases?enrichId=rgreq-1de5ac4c085dd4a641f85dda9c527a37-XXX&enrichSource=Y292ZXJQYWdlOzM1NjgyNTA5NDtBUzoxMTQzMTI4MTA4MzQ2MzY1M0AxNjYyNTk3MzM3NTUz&el=1_x_3&_esc=publicationCoverPdf


Its source code can be found here: https://github.com/Likhitha-palla/UPFP

In [ ]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import glob
import os
import matplotlib
import sortednp
import datetime
import json
from operator import itemgetter
import timeit
import time
import numba

# Algorithm

## Find frequent items (1-pattern)

Both sum of prob and max of time difference (period)

In [ ]:
exclude_columns = ['Time']

In [ ]:
def prepare_database(database_df,
                    min_support, max_period):
    expsup_one_df = database_df.sum(axis=0)

    period_one_df = expsup_one_df.copy(deep=True)
    min_time, max_time = database_df['Time'].min(), database_df['Time'].max()
    for col in database_df.columns:
        if (col == 'Time'): continue
        ser = database_df['Time'][ database_df[col] > 0 ]
        #print(ser.min(), ser.diff().max(), ser.max())
        period_one_df[col] = max( ser.min() - min_time, ser.diff().max(), max_time - ser.max() )

    pf_one_items = [ [col, expsup_one_df[col], period_one_df[col]] 
                        for col in database_df.columns 
                        if (col != 'Time') and (expsup_one_df[col] >= min_support) and (max_period >= period_one_df[col])  ]
    pf_one_df = pd.DataFrame(pf_one_items, columns = ['Item', 'ExpSup', 'MaxPeriod'])
    pf_one_df.sort_values(by=['ExpSup', 'MaxPeriod'], ignore_index=True, ascending=False, inplace=True)
    pf_one_df

    items = pf_one_df['Item'].unique().tolist()
    mining_df = database_df[ ['Time'] + items ]

    #print('Items = ', items)
    return mining_df, items, pf_one_df, min_time, max_time

## Define tree

In [ ]:
class Node(object):
    def __init__(self, item, children, parent, probability=0):
        self.item = item
        self.probability = probability              # expSupCap of the path/pattern
        self.children = children
        self.parent = parent
        #self.times = []
        self.times = np.ndarray(shape=(0))

    def addChild(self, node):
        if (node.item not in self.children):
            self.children[node.item] = []
        self.children[node.item] = node
        node.parent = self

In [ ]:
class Tree(object):
    def __init__(self, items):
        self.nodecounts = 0
        self.root = Node(-1, {}, None)
        self.items = items
        self.nodelists = {}     # item  : [ nodes of this item ]
        for i in items:
            self.nodelists[i] = []
        # self.nodelists = [ [] for i in range(len(items)) ]
        

## Build tree

In [ ]:
def build_tree(matrix, time_col, items, algo_mode: int = 1):
    tree = Tree(items)    
    print("Algo mode = ", algo_mode, " (1 = min, 2 = max)")
    
    x_indexes, y_indexes = np.where( matrix > 0 )
    x_idx = 0
    y_idx = 0
    i = 0
    node = tree.root
    item = items[y_indexes[0]]
    expCap = 1
        
        
    while (i < x_indexes.shape[0]):
        x_idx = x_indexes[i]        # row
        y_idx = y_indexes[i]        # col

        item = items[y_idx]
        #item = y_idx                # now use the index as the item itself (the list of item names are present in the tree already)
        prob = matrix[x_idx][y_idx]
        
        if (item in node.children):
            node = node.children[item]
        else:
            new_node = Node(item, {}, node)
            tree.nodecounts += 1
            node.addChild(new_node)
            node = new_node
            tree.nodelists[item].append(node)
        node.probability += prob*expCap                # multiply by maximum prob of previous items in this transaction.
        
        if (algo_mode == 1):
            expCap = min(expCap, prob)
        elif (algo_mode == 2):
            expCap = max(expCap, prob)                    # changing to max, see if it's better
    
        i += 1
        if (i >= x_indexes.shape[0]) or (x_idx != x_indexes[i]):               # the current row is done, the node is the leaf
            node.times = np.append( node.times, time_col[x_idx] )      
            node = tree.root            # reset back to root
            expCap = 1
        #progress_bar.update(1)
    
    #progress_bar.close()
    return tree

### Remove item from tree entirely

In [ ]:
def remove_item(tree: Tree, item):
    for node in tree.nodelists[item]:
        node.parent.times = np.concatenate([node.parent.times, node.times])
        
        #sortednp.merge( node.parent.times, node.times ) 
        #node.parent.times += node.times
        node.parent.children[item] = None
        del node

## UPFP-growth

In [ ]:
def traverse_path(node: Node):
    path = []
    p = node
    while (p.parent.item != -1):
        path.append(p.parent.item)
        p = p.parent
    path.reverse()
    return path

In [ ]:
@numba.jit(nopython=True)
def find_period(times: np.ndarray, min_time: float, max_time: float):
    if (times is None) or (len(times) == 0): return 9999999
    ts = np.sort(times)
    period = max( min(ts) - min_time, max_time - max(ts) )
    if (len(ts) > 1):
        period = max(period, np.max(np.diff(ts)))
    # for i in range(len(ts)-1):
    #     difference = ts[i+1] - ts[i]
    #     period = max(period, difference)
    return period

### Helper funcs

In [ ]:
def find_frequent_items(cond_patterns, cond_ts, cond_sups, min_support, max_period, min_time, max_time):
    item_sup_per = { }
    for i in range(len(cond_patterns)):
        for item in cond_patterns[i]:
            if (item not in item_sup_per):  
                item_sup_per[item] = [ 0 , np.ndarray(shape=(0)) ]          # support/prob, period
            item_sup_per[item][0] += cond_sups[i]
            item_sup_per[item][1] = np.concatenate( [ item_sup_per[item][1], cond_ts[i] ] )  # appending lists
    for item in item_sup_per:
        item_sup_per[item][1] = find_period(item_sup_per[item][1], min_time, max_time)
    freq_item_dict = { key: value for key, value in item_sup_per.items() if (value[0] >= min_support) and (value[1] <= max_period) }
    freq_item_dict = dict(sorted(freq_item_dict.items(), key=lambda item: (item[1][0], item[1][1])))

    return freq_item_dict

In [ ]:
def make_condition_pattern_base(tree: Tree, item, min_support, max_period, min_time, max_time):
    cond_patterns = [ None for i in range(len(tree.nodelists[item])) ]
    cond_times = [ None for i in range(len(tree.nodelists[item])) ]
    cond_sups = np.ndarray(shape=(len(tree.nodelists[item])))
    #[ None for i in range(len(tree.nodelists[item])) ]
    
    i = 0
    for node in tree.nodelists[item]:
        cond_patterns[i] = traverse_path(node)
        cond_times[i] = node.times 
        cond_sups[i] = node.probability
        i += 1
    freq_item_sup_per = find_frequent_items(cond_patterns, cond_times, cond_sups, min_support, max_period, min_time, max_time)
    
    new_patterns, new_times, new_sups = [], [], np.ndarray(shape=(len(tree.nodelists[item])))
    count = 0
    for p in cond_patterns:
        p1 = [ item for item in p if item in freq_item_sup_per ]
        if (len(p1) > 0):
            p1 = sorted(p1, key=lambda item: (freq_item_sup_per[item][0], freq_item_sup_per[item][1]), reverse=True )
            new_patterns.append(p1)
            new_times.append(cond_times[count])
            #new_sups.append(cond_sups[count])
            new_sups[count] = cond_sups[count]
        count += 1
    new_sups = new_sups[0 : count]
    return freq_item_sup_per, new_patterns, new_times, new_sups

In [ ]:
def add_transaction_condition(tree: Tree,transaction,times,sup):
    node=tree.root
    for item in transaction:
        if item not in node.children:
            new_node=Node(item,{}, node)
            node.addChild(new_node)
            # if item not in tree.nodelists:
            #     tree.nodelists[ item ] = []
            tree.nodelists[item].append(new_node)            
        node = node.children[item] 
        node.probability += sup           
    node.times = np.concatenate( [node.times, times] )

### Main algorithm

In [ ]:
# return the list of all pattern satisfying the constrains. 
def upfp_growth(tree: Tree, prefix, 
                min_support, max_period, 
                min_time, max_time):
    mined_patterns = []

    for i in range(len(tree.items)-1, -1, -1):
        item = tree.items[i]
        newprefix = prefix + [item]
        expsup = 0
        for node in tree.nodelists[item]:
            expsup += node.probability
        if (expsup >= min_support):
            freq_item_sup_per, cond_patterns, cond_times, cond_sups = make_condition_pattern_base(tree, item, 
                                                                                min_support, max_period, 
                                                                                min_time, max_time)
            cond_tree = Tree(list(freq_item_sup_per.keys()))
            for p in range(len(cond_patterns)):
                add_transaction_condition(cond_tree, cond_patterns[p], cond_times[p], cond_sups[p])
            if (len(cond_patterns) > 0):
                mined_patterns += upfp_growth(cond_tree, newprefix, min_support, max_period, min_time, max_time ) 
            else:
                # if no more items to search: stop and return.
                mined_patterns.append(np.array(newprefix, dtype=np.int16))
        remove_item(tree, item)
    return mined_patterns

## Recalculate actual Expected support for one more time

In [ ]:
def counting_sort(array, count_func):
    max_count = 0
    for i in range(len(array)):
        max_count = max(max_count, count_func(array[i]))

    # the usable count is [0, max_count], inclusive, hence (max_count + 1) different counts
    # we need h[max_count+1] be the end index of sorted array, for iterating the last count's indexes
    count_list_heads = [ 0 for i in range(max_count + 2)]          
    for i in range(len(array)):
        count_list_heads[ count_func(array[i]) ] += 1
    
    # cumulative sum
    for i in range(max_count + 1):
        count_list_heads[i + 1] += count_list_heads[i]
    
    sorted_array = [ None for i in range(len(array)) ]
    for i in range(len(array)):
        count = count_func(array[i])
        count_list_heads[ count ] -= 1
        sorted_array[ count_list_heads[ count ] ] = array[i] 

    return sorted_array, count_list_heads


In [ ]:
@numba.jit(nopython=True)
def pattern_priority(ptn):
    return np.min(ptn)

In [ ]:
@numba.jit(nopython=True)
def find_expSup(pattern_matrix: np.ndarray):
    probabilities = np.zeros(pattern_matrix.shape[0])
    for i in range(pattern_matrix.shape[0]):
        probabilities[i] = np.prod(pattern_matrix[i, :])
    # probabilities = np.prod(pattern_matrix, axis=1, keepdim=False)
    return probabilities, np.sum(probabilities)

In [ ]:
@numba.jit(nopython=True)
def find_final_period(times: np.ndarray, 
                    probabilities: np.ndarray, 
                    min_time: float, max_time: float):
    period = 0
    times = times.copy()
    times = times[ probabilities > 0 ]
    # times = times.to('cuda:0')
    if (times is None) or (times.shape[0] == 0): 
        period = 999999
    else:
        period = max( np.min(times) - min_time, max_time - np.max(times) )
    if (len(times) > 1):
        period = max( period, np.max(np.diff(times)) )
    return period

In [ ]:
def filter_patterns(mined_patterns, mining_df, min_time, max_time, min_support, max_period):
    
    sorted_patterns, length_heads = counting_sort(mined_patterns, lambda ptn: len(ptn))
    for length in range(1, len(length_heads) - 2):
        sorted_patterns[ length_heads[length] : length_heads[length+1] ] = sorted(
                                    sorted_patterns[length_heads[length] : length_heads[length+1]],
                                    key = lambda p: pattern_priority(p) )

    filtered_patterns = []
    print('Potential max pattern length = ', len(length_heads) - 2)
    
    tsr = mining_df.drop(columns=['Time'], axis=1).to_numpy()
    time_tsr = mining_df['Time'].to_numpy()
    #tsr = torch.FloatTensor( mining_df.drop(columns=['Time'], axis=1).to_numpy() ).detach().to(device)      # for faster calculation for big dataset
    #time_tsr = torch.LongTensor( mining_df['Time'].to_numpy()).detach().to(device)
    

    # binary search
    min_length = 1
    max_length = len(length_heads) - 2
    length = 0
    start_pattern_idx = -1
    max_pattern_start_idx = -1


    # quick search to reduce the space needed to scan.
    length = 1
    while (length <= max_length):
        pattern_idx = length_heads[length]
        start_pattern_idx = -1
        while (pattern_idx < length_heads[length + 1]):
            ptn = sorted_patterns[pattern_idx]  
            #sub_tsr = torch.index_select(input=tsr, dim=1, index=torch.LongTensor(ptn).to(device))
            sub_tsr = tsr[ :, ptn ]
            probabilities, expSup = find_expSup(sub_tsr)
            period = find_final_period(time_tsr, probabilities,
                                min_time, max_time)
            if (expSup >= min_support) and (period <= max_period):       
                start_pattern_idx = pattern_idx
                break
            #torch.cuda.empty_cache() 
            if (length_heads[length + 1] - length_heads[length] == 1):
                break
            pattern_idx += int((np.log2(length_heads[length + 1] - length_heads[length])))
            
        if (start_pattern_idx > -1):
            print("Found the pattern with length = ", length)
            max_pattern_length = length
            max_pattern_start_idx = length_heads[length]
            min_length = length + 1
        length *= 2


    # this search find the most possible length, but you must pass it another time to have all patterns with such length.
    while (min_length <= max_length):
        start_pattern_idx = -1
        length = (min_length + max_length) // 2
        print("Checking potential patterns with length = ", length)

        for pattern_idx in tqdm(range( length_heads[length], length_heads[ length + 1 ]), desc="pattern" ):
            ptn = sorted_patterns[pattern_idx]
            sub_tsr = tsr[ :, ptn ]
            probabilities, expSup = find_expSup(sub_tsr)
            period = find_final_period(time_tsr, probabilities,
                                min_time, max_time)

            if (expSup >= min_support) and (period <= max_period):       
                start_pattern_idx = pattern_idx
                break

        if (start_pattern_idx > -1):
            print("Found the pattern with length = ", length)
            max_pattern_length = length
            max_pattern_start_idx = start_pattern_idx
            min_length = length + 1
        else:
            max_length = length - 1
        

    print("Now calculating all patterns with max length...")
    for pattern_idx in tqdm(range( max_pattern_start_idx, length_heads[ max_pattern_length + 1 ]), desc="pattern" ):
        ptn = sorted_patterns[pattern_idx]
        sub_tsr = tsr[ :, ptn ]
        probabilities, expSup = find_expSup(sub_tsr)
        period = find_final_period(time_tsr, probabilities,
                            min_time, max_time)
        if (expSup >= min_support) and (period <= max_period):          
            filtered_patterns.append( [ ptn, expSup, period ] ) 
        

    filtered_patterns.sort( key = lambda p: (len(p[0]), -p[2], p[1]), reverse=True )
    return filtered_patterns, max_pattern_length, len(length_heads) - 2

In [ ]:
def make_strings_from_patterns(patterns, item_list):
    pattern_strings = []
    for ptn in patterns:
        item_str = ""
        for i in range(len(ptn[0]) - 1):
            item_str += item_list[ptn[0][i]] + ', '
        item_str += item_list[ptn[0][ len(ptn[0]) - 1 ]]
        item_str += ':[' + str(ptn[1]) + ', ' + str(ptn[2]) + ']\n' 
        pattern_strings.append( item_str )
    return pattern_strings

In [ ]:
def produce_patterns(mined_patterns, mining_df, item_list, min_time, max_time, min_support, max_period):

    final_patterns, max_pattern_length, potential_max_length = filter_patterns(mined_patterns, mining_df, 
                                min_time, max_time, min_support, max_period)
    #print('Max pattern length: ', max_pattern_length)
    pattern_strings = make_strings_from_patterns(final_patterns, item_list)
    if (len(pattern_strings) > 0): print('Most periodic pattern = ', pattern_strings[0])
    return pattern_strings, final_patterns, max_pattern_length, potential_max_length

## Export results

In [ ]:
def export_results(output_folder_path: str, 
                pattern_strings, pf_one_df, database_path, min_support, max_period):
    os.makedirs(output_folder_path, exist_ok=True)
    pf_one_df.to_csv(os.path.join(output_folder_path,'fp_pattern_one.csv'))
    with open(os.path.join(output_folder_path, 'patterns.txt'), 'w') as f:
        f.writelines(pattern_strings)
        f.close()
    with open(os.path.join(output_folder_path, 'setting.json'), 'w') as f:
        f.write( json.dumps({ 'data_path': database_path,  'min_support': min_support, 'max_period': max_period }, indent=4) )

## Main functions

In [ ]:
timetable_columns = [ 'Mode', 'MinSup', 'MaxPer', 
                'PFItemCount', 
                'prepare_database', 'make_transaction', 'build_tree', 'upfp_growth', 'produce_pattern', 
                'max_pattern_length', 'potential_max_length', 'potential_pattern_count',
                'size_tree' ]

In [ ]:
def mine_patterns(database_df, output_folder_path, 
                min_support, max_period, algo_mode: int = 1, export: bool = True):
    
    times = [ 0 for i in range(5) ]
    
    times[0] = timeit.default_timer()
    mining_df, item_list, pf_one_df, min_time, max_time = prepare_database(database_df, min_support, max_period)
    times[0] = timeit.default_timer() - times[0]
    
    if (len(item_list) == 0): 
        return [], pf_one_df, None
        
    times[2] = timeit.default_timer()
    #tree = build_tree(transactions, item_list, algo_mode)
    time_col = mining_df['Time'].copy().to_numpy()         # to ensure no editing shall affect the main data
    npmatrix = mining_df.drop(columns=['Time'], axis=1).to_numpy()
    tree = build_tree(npmatrix, time_col, list(range(len(item_list))), algo_mode)
    times[2] = timeit.default_timer() - times[2]
    size_tree = tree.nodecounts
    print("Tree node count = ", size_tree)

    times[3] = timeit.default_timer()
    mined_patterns = upfp_growth(tree, [], min_support, max_period, min_time, max_time )
    times[3] = timeit.default_timer() - times[3]
    potential_pattern_counts = len(mined_patterns)
    print("Algo time = ", times[3])

    max_pattern_length, potential_max_length = -1, -1
    times[4] = timeit.default_timer()
    if (export):
        pattern_strings, _, max_pattern_length, potential_max_length = produce_patterns(mined_patterns, mining_df, item_list,
                                                            min_time, max_time, min_support, max_period)
        export_results(output_folder_path, pattern_strings, pf_one_df, database_path, min_support, max_period)
    else: pattern_strings = ""
    times[4] = timeit.default_timer() - times[4]

    times = [algo_mode, min_support, max_period, len(item_list) ] + times + [max_pattern_length, potential_max_length, potential_pattern_counts, size_tree]
    
    return pattern_strings, pf_one_df, times

# Input

In [ ]:
dataset_name = 'UrbanAir'
database_path = r"../data/UTDATABASE/utd_20221222_0226/database.csv"
database_df = pd.read_csv(database_path)
database_df.fillna(0, inplace=True)
# database_df.describe()

# Full data run

In [ ]:
min_default_support = 500
max_default_period = 1000

In [ ]:
output_folder_path = f'../output_temp/{dataset_name}/FULLDATA/ufp_' + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
os.makedirs(output_folder_path, exist_ok=True)
mine_patterns(database_df, output_folder_path, min_default_support, max_default_period)

# With only traffic or sensor data

## Define constants

In [ ]:
sensor_columns = ['Altitude', 'Temperature', 'Humidity', 'Rainfall', 'WindGust', 'WindSpeed', 'WindCos', 'WindSin', 'UV']
traffic_columns = ['Person', 'Car', 'Bus', 'Motorbike', 'Truck']

In [ ]:
def delete_columns(database_df, columns):
    column_names = []
    for c in database_df.columns:
        for l in columns:
            if l in c:
                column_names.append(c)
                break
    #print(column_names)
    new_database_df = database_df.drop( column_names, axis=1 ).reset_index(drop=True)
    #new_database_df.info()
    return new_database_df

## Mining with only traffic data

In [ ]:
min_traffic_support = 10
max_traffic_period = 1000

In [ ]:
output_traffic_folder_path = f'../output_temp/{dataset_name}/TRAFFIC/ufp_traffic_' + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
os.makedirs(output_traffic_folder_path, exist_ok=True)

traffic_database_df = delete_columns(database_df, sensor_columns)
mine_patterns(traffic_database_df, output_traffic_folder_path,
            min_traffic_support, max_traffic_period)
# del traffic_database_df

## Mining with only sensor data

In [ ]:
# output_sensor_folder_path = '../data/UTDatabase/Mining/ufp_sensor_' + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
# os.makedirs(output_sensor_folder_path, exist_ok=True)

# sensor_database_df = delete_columns(database_df, traffic_columns)
# mine_patterns(sensor_database_df, output_sensor_folder_path, min_support, max_period)
# del sensor_database_df

# Customized mining

## Mining with specific columns for traffic relation finding

This is to see if for extreme cases such as PM10 lv4, is there a hidden pattern among them?
Each columns shall be selected, and the dataframe will be filtered based on that col.

In [ ]:
min_spec_support = 3
max_spec_period = 2000

In [ ]:
traffic_database_df = delete_columns(database_df, sensor_columns)

In [ ]:
# exclude_unhelpful_columns = ['CO', 'NO2', 'SO2']
# dropping_unhelpful_columns = []
# for col in traffic_database_df.columns:
#     for ex in exclude_unhelpful_columns:
#         if (ex in col) :
#             dropping_unhelpful_columns.append(col)
# new_traffic_database_df = traffic_database_df.drop( dropping_unhelpful_columns , axis=1).reset_index(drop=True)

In [ ]:
# for col in new_traffic_database_df.columns:
#     if ('HIGH' in col) or ('lv4' in col) or ('lv5' in col) or ('HourTriple' in col): 
#         sub_df = new_traffic_database_df[ new_traffic_database_df[col] > 0 ].copy(True).reset_index(drop=True)
#         output_col_path = os.path.join(output_speccol_path, col)
#         mine_patterns(sub_df, output_col_path, min_spec_support, max_spec_period )
#         del sub_df

## Mining with each one AQI item

For each iteration, isolate one AQI item from the rest (delete all other AQI items), to find pattern specifically for that item.

In [ ]:
min_AQI_support = 1
max_AQI_period = 2000

In [ ]:
aqi_label_columns = [ col for col in traffic_database_df.columns if 'AQI' in col ]
aqi_label_columns

aqi_df = traffic_database_df[ aqi_label_columns ].copy(True)

custom_aqi_database_df = traffic_database_df.drop( aqi_label_columns, axis=1 )
custom_aqi_database_df = custom_aqi_database_df.drop( [ c for c in custom_aqi_database_df.columns if 'WeekDay' in c ], axis=1 )

In [ ]:
output_aqi_path = f"../output_temp/{dataset_name}/AQI/ufp_aqi_" + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
os.makedirs(output_aqi_path, exist_ok=True)

for aqicol in aqi_label_columns:
    print(aqicol)
    sub_df = custom_aqi_database_df.copy(True)
    sub_df[aqicol] = aqi_df[aqicol].copy(True)
    sub_df = sub_df[ sub_df[aqicol] > 0 ].sort_values(by=['Time'], ignore_index=True)               # prob = 0 mean not exist, dont use
    if (sub_df.shape[0] == 0): continue
    output_col_path = os.path.join(output_aqi_path, aqicol)
    mine_patterns(sub_df, output_col_path, min_AQI_support, max_AQI_period )
    del sub_df

## Mining with only high traffic signals

In [ ]:
min_hightraffic_support = 15
max_hightraffic_period = 2000

In [ ]:
# high_traffic_database_df = traffic_database_df
# for col in traffic_columns:
#     high_traffic_database_df = high_traffic_database_df[ high_traffic_database_df[ col + '_LOW' ] == 0 ]
# high_traffic_database_df.reset_index(drop=True, inplace=True)

In [ ]:
# output_hightraffic_path = r"../output_temp/HIGHTRAFFIC/ufp_hightraffic_" + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
# os.makedirs(output_hightraffic_path, exist_ok=True)

# mine_patterns(high_traffic_database_df, output_hightraffic_path, min_hightraffic_support, max_hightraffic_period)

# Mining with each traffic item

The same process to each AQI item.

In [ ]:
min_eachtraffic_support = 1.5
max_eachtraffic_period = 1000

In [ ]:
traffic_label_columns = []
for tcol in traffic_columns:
    for col in traffic_database_df.columns:
        if (tcol in col):
            traffic_label_columns.append(col)

trafficonly_df = traffic_database_df[ traffic_label_columns ].copy(True)
traffic_excluded_database_df = traffic_database_df.drop( traffic_label_columns, axis=1 )
# traffic_excluded_database_df = traffic_excluded_database_df.drop( [ c for c in traffic_excluded_database_df.columns if 'WeekDay' in c ], axis=1 )

In [ ]:
traffic_excluded_database_df.describe()

In [ ]:
output_eachtraffic_path = f"../output_temp/{dataset_name}/EACHTRAFFIC/ufp_eachtraffic_" + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
os.makedirs(output_eachtraffic_path, exist_ok=True)

for c in traffic_label_columns:
    print(c)
    sub_df = traffic_excluded_database_df.copy(True)
    sub_df[c] = trafficonly_df[c].copy(True)
    sub_df = sub_df[ sub_df[c] > 0 ].sort_values(by=['Time'], ignore_index=True)               # prob = 0 mean not exist, dont use
    if (sub_df.shape[0] == 0): continue
    output_col_path = os.path.join(output_eachtraffic_path, c)
    mine_patterns(sub_df, output_col_path, min_eachtraffic_support, max_eachtraffic_period )
    del sub_df